In [1]:
!pip install pandas
!pip install graphdatascience

In [2]:
from graphdatascience import GraphDataScience

/home/pkedia/miniconda3/envs/cfde/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_AUTH = (
    "neo4j",
    "Hello@001"
)
NEO4J_DB = "neo4j"

gds = GraphDataScience(NEO4J_URI, auth = NEO4J_AUTH, database = NEO4J_DB)

In [4]:
from graphdatascience.server_version.server_version import ServerVersion
assert gds.server_version() >= ServerVersion(2,3,0)

In [1]:
"""
Required dataframe type for nodes and relationships

nodes = pandas.DataFrame(
    {
        "nodeId": [0, 1, 2, 3],
        "labels":  ["A", "B", "C", "A"],
        "prop1": [42, 1337, 8, 0],
        "otherProperty": [0.1, 0.2, 0.3, 0.4]
    }
)

relationships = pandas.DataFrame(
    {
        "sourceNodeId": [0, 1, 2, 3],
        "targetNodeId": [1, 2, 3, 0],
        "relationshipType": ["REL", "REL", "REL", "REL"],
        "weight": [0.0, 0.0, 0.1, 42.0]
    }
)

"""

'\nRequired dataframe type for nodes and relationships\n\nnodes = pandas.DataFrame(\n    {\n        "nodeId": [0, 1, 2, 3],\n        "labels":  ["A", "B", "C", "A"],\n        "prop1": [42, 1337, 8, 0],\n        "otherProperty": [0.1, 0.2, 0.3, 0.4]\n    }\n)\n\nrelationships = pandas.DataFrame(\n    {\n        "sourceNodeId": [0, 1, 2, 3],\n        "targetNodeId": [1, 2, 3, 0],\n        "relationshipType": ["REL", "REL", "REL", "REL"],\n        "weight": [0.0, 0.0, 0.1, 42.0]\n    }\n)\n\n'

In [26]:
import pandas as pd

park_df = pd.read_csv("parkinson_trunc.csv")
park_df.head(10)

,positive_genes_cui,positive_genes_label,features,features_to_remove,unknown_genes_cui,unknown_genes_label,pos_ids,unk_ids,r1,r2
0,C1414209,DYSF gene,C0023212,C0005578,C1412137,ACTA2 gene,714968,706646,associated_with,associated_with
1,C1414209,DYSF gene,C0023212,C0005578,C1419216,KIF20A gene,714968,709722,associated_with,associated_with
2,C1414209,DYSF gene,C0023212,C0005578,C1423020,GTPBP3 gene,714968,732531,associated_with,associated_with
3,C1414209,DYSF gene,C0023212,C0005578,C1414899,GAA gene,714968,695327,associated_with,associated_with
4,C1414209,DYSF gene,C0023212,C0005578,C1826654,PPCS gene,714968,713476,associated_with,associated_with
5,C1414209,DYSF gene,C0023212,C0005578,C1420009,SGCG gene,714968,729483,associated_with,associated_with
6,C1414209,DYSF gene,C0023212,C0005578,C1537558,LDLRAP1 gene,714968,723232,associated_with,associated_with
7,C1414209,DYSF gene,C0023212,C0005578,C1425226,FKRP gene,714968,730197,associated_with,associated_with
8,C1414209,DYSF gene,C0023212,C0005578,C1422255,ABCG5 gene,714968,707395,associated_with,associated_with
9,C1414209,DYSF gene,C0023212,C0005578,C1422256,ABCG8 gene,714968,717726,associated_with,associated_with


In [34]:
unique_pos_cui = park_df[["positive_genes_cui", "pos_ids"]].drop_duplicates(["positive_genes_cui"])
unique_unk_cui = park_df[["unknown_genes_cui", "unk_ids"]].drop_duplicates(["unknown_genes_cui"])
unique_features = park_df["features"].drop_duplicates()
# pos_ids = park_df[park_df["positive_genes_cui"] == unique_pos_cui]
print(unique_pos_cui.head(), unique_pos_cui.shape)
print(unique_features.shape)

pos_labels = ["1" for _ in range(unique_pos_cui.shape[0])]
unk_labels = ["0" for _ in range(unique_unk_cui.shape[0])]

cuis = pd.DataFrame({
    "cui": pd.concat([unique_pos_cui["pos_ids"], unique_unk_cui["unk_ids"]]),
    "label": pos_labels + unk_labels
})

cuis.drop_duplicates(["cui"], inplace = True)
print(cuis.shape)

  positive_genes_cui  pos_ids
0           C1414209   714968 (1, 2)
(4,)
(324, 2)


In [35]:
def features_present(row, features):
    return {feature: 1 if feature in row["features"] else 0 for feature in features}

features_df = park_df.apply(features_present, axis =1, result_type = "expand", args = (unique_features,))
features_df.head()

,C0023212,C4020858,C0700251,C0686353
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [36]:
nodes = pd.DataFrame(
    {
        "nodeId": cuis["cui"],
        "labels":  cuis["label"],
    }
)
nodes[features_df.columns] = features_df
# nodes[new_df]
print(nodes)


     nodeId labels  C0023212  C4020858  C0700251  C0686353
0    714968      1         1         0         0         0
0    706646      0         1         0         0         0
1    709722      0         1         0         0         0
2    732531      0         1         0         0         0
3    695327      0         1         0         0         0
..      ...    ...       ...       ...       ...       ...
640  505183      0         0         0         0         1
641  707352      0         0         0         0         1
644  703299      0         0         0         0         1
646  708707      0         0         0         0         1
648  452214      0         0         0         0         1

[324 rows x 6 columns]


In [37]:
relationships = pd.DataFrame(
    {
        "sourceNodeId": park_df["pos_ids"],
        "targetNodeId": park_df["unk_ids"],
        "relationshipType": park_df["r1"]
    }
)

In [40]:
GRAPH_NAME = "gene-gnn"
PIPELINE_NAME = "gene-pipeline"

In [59]:
if gds.graph.exists(GRAPH_NAME).exists:
    gds.graph.drop(GRAPH_NAME)

G = gds.graph.construct(
    GRAPH_NAME,      # Graph name
    nodes,           # One or more dataframes containing node data
    relationships    # One or more dataframes containing relationship data
)

In [60]:
print(G.node_labels(), G.relationship_types(), G.node_properties())

['1', '0'] ['disease_has_associated_gene', 'gene_associated_with_disease', 'inverse_associated_with', 'associated_with'] 1    [C0700251, C0023212, C0686353, C4020858]
0    [C0700251, C0023212, C0686353, C4020858]
dtype: object


In [62]:
if gds.pipeline.exists(PIPELINE_NAME).exists:
    gds.pipeline.drop(PIPELINE_NAME)
    
pipe, _ = gds.beta.pipeline.nodeClassification.create(PIPELINE_NAME)
    
_ = pipe.configureSplit(testFraction = 0.796)

In [63]:
_ = pipe.addNodeProperty(
    "beta.hashgnn",
    mutateProperty="embedding",
    iterations = 4, 
    heterogeneous = True, 
    embeddingDensity = 512, 
    neighborInfluence = 0.7,
    featureProperties = unique_features,
    randomSeed = 42,
    contextNodeLabels = G.node_labels()
)

In [64]:
_ = pipe.selectFeatures("embedding")

In [65]:
_ = pipe.addLogisticRegression(penalty = (0.1, 1.0), maxEpochs = 1000, patience = 5, tolerance = 0.0001, learningRate = 0.01)
_ = pipe.addRandomForest(minSplitSize = (2, 100), criterion = "ENTROPY")

In [81]:
if gds.model.exists("gene-model").exists:
    gds.beta.model.drop("gene-model")
model, _ = pipe.train(
    G,
    modelName = "gene-model",
    targetNodeLabels = ["1", "0"],
    targetProperty = "C0686353",
    metrics = ["F1_MACRO", "ACCURACY"],
    randomSeed = 42
)

Node Classification Train Pipeline: 100%|███| 100.0/100 [00:02<00:00, 34.28%/s]


In [82]:
model.metrics()

F1_MACRO    {'train': {'min': 0.488372090411033, 'avg': 0....
ACCURACY    {'train': {'min': 0.95454546, 'avg': 0.9545454...
dtype: object